# References
- How to get GPT2 345M: https://github.com/huggingface/pytorch-pretrained-BERT/issues/582
- https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_gpt2.py

In [1]:
from functools import partial
from tqdm import trange
import torch
import torch.nn.functional as F
import numpy as np
import pytorch_pretrained_bert
from pytorch_pretrained_bert import GPT2LMHeadModel, GPT2Tokenizer
for mod in (np, torch, pytorch_pretrained_bert):
    print(f'{mod.__name__}: {mod.__version__}')

numpy: 1.16.2
torch: 1.1.0
pytorch_pretrained_bert: 0.6.2


# Build model GPT2

In [2]:
seed = 0
np.random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name_or_path = 'gpt2'
model_name_or_path = 'torch_GPT2-345M/'
enc = GPT2Tokenizer.from_pretrained('gpt2')

model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (h): ModuleList(
      (0): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (1): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (2): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (3): Block(
        (ln_1): BertLayerNorm()
    

In [3]:
line = "Dummy"
tokens_tensor = torch.tensor(enc.encode(line)).reshape(1, -1)
predictions, _ = model(tokens_tensor)
vocab_size = predictions.shape[-1]
assert vocab_size == 50257  # GPT2 vocab size is 50257

In [5]:
predictions.shape, tokens_tensor.shape

(torch.Size([1, 2, 50257]), torch.Size([1, 2]))

# Minimal example

In [8]:
batch_size=1
context_tokens = enc.encode(prompt)
context = torch.tensor(context_tokens, device=device, dtype=torch.long).unsqueeze(0).repeat(batch_size, 1)
context

tensor([[   34,   945,   547, 15646,   287]])

In [11]:
top_k-1

1

In [15]:
context_tensor

tensor([[   34,   945,   547, 15646,   287]])

In [16]:
predicted_index

tensor(198)

In [19]:
prompt = "Cars were invented in"
max_predictions = 16
top_k = 2

context_tokens = enc.encode(prompt)
context_tensor = torch.tensor(context_tokens, dtype=torch.long).reshape(1, -1)

prev = context_tensor
output = context_tensor.clone()
past = None
for i in range(max_predictions):
    #predictions, past = model(output, past=past)
    predictions, past = model(prev, past=past)
    context_size = prev.shape[1]
    assert predictions.shape == (1, context_size, vocab_size)
    
    last_prediction = predictions[:, -1, :]
    topk = torch.topk(last_prediction, 10)
    prev = torch.tensor([[ topk.indices[0][top_k-1] ]])
    output = torch.cat((output, prev), dim=-1)
    
enc.decode(output[0].tolist())

'Cars were invented in 1876.\nThe car is the first car that was ever designed and manufactured'

> **NOTE**: the first iteration `prev` is the context tensor and `past` is None. From the second iteration,
> `prev` is just the last predicted token and all the history is stored in `past`.

GPT2 345M has 24 layers and d_model = 1024

In [25]:
64*16

1024

In [24]:
type(past), len(past), output.shape

(list, 24, torch.Size([1, 21]))

In [22]:
[t.shape for t in past]

[torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64]),
 torch.Size([2, 1, 16, 20, 64])]

# Minimal example with sampling

In [29]:
seed = 0
prompt = "Cars were invented in"
max_predictions = 16
top_k = 40

np.random.seed(seed)
torch.random.manual_seed(seed)
context_tokens = enc.encode(prompt)
context_tensor = torch.tensor(context_tokens, dtype=torch.long).reshape(1, -1)

prev = context_tensor
output = context_tensor.clone()
past = None
for i in range(max_predictions):
    #predictions, past = model(output, past=past)
    predictions, past = model(prev, past=past)
    context_size = prev.shape[1]
    assert predictions.shape == (1, context_size, vocab_size)
    
    last_prediction = predictions[:, -1, :]
    topk = torch.topk(last_prediction, 10)   
    log_probs = F.softmax(topk.values, dim=-1)  # softmax among the top-k
    rand_idx_in_topk = torch.multinomial(log_probs, num_samples=1)
    predicted_index = topk.indices[0][rand_idx_in_topk]      
    
    prev = torch.tensor([[ predicted_index ]])
    output = torch.cat((output, prev), dim=-1)
    
print(enc.decode(output[0].tolist()))

Cars were invented in Britain and are owned by many people in Britain. The cars are used for driving


# Explore model

In [51]:
%run -i textgen.py

In [52]:
decoder = partial(decoder_gpt2, enc=enc)

In [53]:
gen_text = partial(generate_text_gpt2, model, enc, decoder)
#gen_text?

In [54]:
gen_text('What do you know about Machine Learning and Natural Language Processing?', 
         length=10, seed=0, sample=True, top_k=10)

  0%|          | 0/10 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 466, 345, 760, 546, 10850, 18252, 290, 12068, 15417, 28403, 30]


100%|██████████| 10/10 [00:02<00:00,  4.85it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Share your thoughts below.<|endoftext|>


In [55]:
gen_text('What do you know about Machine Learning and Natural Language Processing?', 
         length=130, seed=1, sample=True, top_k=10)

  0%|          | 0/130 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 466, 345, 760, 546, 10850, 18252, 290, 12068, 15417, 28403, 30]


100%|██████████| 130/130 [00:21<00:00,  6.35it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================


I am an expert in artificial intelligence and natural language processing.

What is your background and expertise in Artificial Intelligence?

I am a graduate from MIT Media Lab where I worked on the machine learning team.

What is your favorite thing about artificial intelligence?

My favorite thing about machine learning is that it's so easy. It's so easy to understand. It's a very intuitive thing to understand. Machine learning is really easy to understand as well. If you understand machine learning, you'll understand how to apply it for different tasks, and that makes it so easy to learn how to use it.

How


In [55]:
gen_text('What do you know about Machine Learning and Natural Language Processing?', 
         length=130, seed=1, sample=True, top_k=10)

  0%|          | 0/130 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 466, 345, 760, 546, 10850, 18252, 290, 12068, 15417, 28403, 30]


100%|██████████| 130/130 [00:23<00:00,  5.63it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================


I am an expert in artificial intelligence and natural language processing.

What is your background and expertise in Artificial Intelligence?

I am a graduate from MIT Media Lab where I worked on the machine learning team.

What is your favorite thing about artificial intelligence?

My favorite thing about machine learning is that it's so easy. It's so easy to understand. It's a very intuitive thing to understand. Machine learning is really easy to understand as well. If you understand machine learning, you'll understand how to apply it for different tasks, and that makes it so easy to learn how to use it.

How


In [58]:
gen_text('What do you know about Machine Learning and Natural Language Processing?', 
         length=200, seed=2, sample=True, top_k=10)

  0%|          | 0/200 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 466, 345, 760, 546, 10850, 18252, 290, 12068, 15417, 28403, 30]


100%|██████████| 200/200 [00:35<00:00,  4.99it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Take a look at these links.

What is Machine Learning and Natural Language Processing?

Machine Learning is the process of gathering, processing, and understanding data. This can be applied to a wide range of fields of research. The term "Natural Language Processing" is used to describe the process by which computer programs are used to learn and recognize information. Machine Learning refers to the process of gathering, processing, and understanding data and can be applied to all areas of computer science. It is one of the most popular areas of research, with a large amount of data.

Machine Learning, as well as its related applications such as Machine Learning-as-a-Service (MLA-as-a-Service) and Artificial Intelligence, are often considered "technologies" and thus have been treated in different ways. For the purpo

In [59]:
gen_text('What do you know about Machine Learning and Natural Language Processing?', 
         length=200, seed=2, sample=True, top_k=40)

  0%|          | 0/200 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 466, 345, 760, 546, 10850, 18252, 290, 12068, 15417, 28403, 30]


100%|██████████| 200/200 [00:35<00:00,  5.33it/s]


PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Tell us about it by hitting the comments below, and keep your eye out for updates.<|endoftext|>


In [13]:
gen_text('What is Machine Learning and Natural Language Processing?\n\nMachine Learning is', 
         length=200, seed=3, sample=True, top_p=0.5)

  0%|          | 0/200 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 318, 10850, 18252, 290, 12068, 15417, 28403, 30, 198, 198, 37573, 18252, 318]


100%|██████████| 200/200 [00:39<00:00,  5.08it/s]

PROMPT: What is Machine Learning and Natural Language Processing?

Machine Learning is
======================================== SAMPLE 1 ========================================
 a field from which humans make untapped skills like image recognition, speech synthesis, and criminal profiling. Once found, these skills are often applied in machine learning-in-situ (MLI), where human researchers attempt to image how information is being processed in different kinds of machines. This process can be quite an art, just to hone even the most basic skills involved like common sense.

Natural language processing concerns algorithms, like natural speech recognition, who learn to understand language through words. It's a natural parallel with machine learning, which is how whether you describe running up a escalator or rustling feet prepared you for walking battering rams. It's not that at this level of effectiveness all scientists should be making their living as judges for nightmarish videos. Mac

In [18]:
prompt = """What is Machine Learning and Natural Language Processing?

Natural Language Processing is a branch of Machine Learning """ 
gen_text(prompt,
         length=150, seed=3, sample=True, top_p=0.9)

  0%|          | 0/150 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 318, 10850, 18252, 290, 12068, 15417, 28403, 30, 198, 198, 35364, 15417, 28403, 318, 257, 8478, 286, 10850, 18252, 220]


100%|██████████| 150/150 [00:26<00:00,  4.92it/s]

PROMPT: What is Machine Learning and Natural Language Processing?

Natural Language Processing is a branch of Machine Learning 
======================================== SAMPLE 1 ========================================
̻ library. Machine Learning is the type of technology which uses multiple computational resources to teach artificial intelligence ※ a kind of program or software which shapes itself.

In Machine Learning, one tries to create "robots" which perform many different tasks in order to learn new things. This way, we are using different fields, Neural Networks, Algorithms etcetera to create a "machine" which learns complex, difficult and rewarding tasks. A machine that can recognize pictures in a room and use its manual interpretation to accomplish the task, this is called Artificial Neural Network. ̻ Machine Learning is calculated using Deep Learning algorithms and one thing that has to be taken into account is that machine learning requires natural language processing. ̻


In [19]:
prompt = """What is Machine Learning and Natural Language Processing?

Natural Language Processing is a branch of Machine Learning """ 
gen_text(prompt,
         length=150, seed=3, sample=True, top_p=0.1)

  0%|          | 0/150 [00:00<?, ?it/s]

context[<class 'list'>]: [2061, 318, 10850, 18252, 290, 12068, 15417, 28403, 30, 198, 198, 35364, 15417, 28403, 318, 257, 8478, 286, 10850, 18252, 220]


100%|██████████| 150/150 [00:29<00:00,  4.92it/s]

PROMPT: What is Machine Learning and Natural Language Processing?

Natural Language Processing is a branch of Machine Learning 
======================================== SAMPLE 1 ========================================
̻that can produce machine-readable text such as documents, images or other data; processed it into machine-readable text, could then be processed again, a system of classification, tools and neural networks ̻to eventually produce image or other text to produce document ̻or any other data.

There are a hierarchical categories of deep learning types processor labeled for various purposes,

dClassifiers enable

that saves energy and memory by working with Data sets that are very large, such as Texts .

. Latent Classifiers enable the use of RNN based models in Learning tasks or Machine Learning tasks.

See the following video to know more about Machine Learning for Texts:

What are


In [7]:
seed = 0
np.random.seed(seed)
torch.random.manual_seed(seed)
torch.cuda.manual_seed(seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name_or_path = 'gpt2'
model_name_or_path = 'torch_GPT2-345M/'
enc = GPT2Tokenizer.from_pretrained('gpt2')

model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (h): ModuleList(
      (0): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (1): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (2): Block(
        (ln_1): BertLayerNorm()
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
        )
        (ln_2): BertLayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
        )
      )
      (3): Block(
        (ln_1): BertLayerNorm()
    

In [8]:
model.config.n_ctx

1024

In [22]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=125, seed=0, sample=False)

100%|██████████| 125/125 [00:20<00:00,  5.87it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================


I'm a software engineer and I'm currently working on a project called "Machine Learning for the Web". I'm also a big fan of natural language processing and I'm working on a project called "Natural Language Processing for the Web".

What's your favorite thing about machine learning?

I love the fact that it's so easy to use. I love the fact that it's so easy to use.

What's your favorite thing about natural language processing?

I love the fact that it's so easy to use. I love the fact that it's so easy to use.



In [48]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=125, seed=0, sample=True, top_k=40)

100%|██████████| 125/125 [00:24<00:00,  5.10it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Feel free to join us at the 2017 AI conference in San Francisco to learn more about it

This article is part of The Digital Finance Podcast, where we'll be covering topics like AI and finance.

What are you waiting for!? Listen now, and subscribe for more stuff like the episode is in this playlist:

Subscribe to us on iTunes: iTunes, Stitcher, Apple Podcasts, Google Play, SoundCloud, Soundcloud Stitcher RSS, RSS: The digital finance podcast

Check out other podcasts you might enjoy:<|endoftext|>


In [53]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=125, seed=0, sample=True, top_p=0.9, nsamples=4)

  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Skip all the introductory materials we make available (get up to speed by reading about these areas). Take a Google or TechNet course on Machine Learning. Learn how to use Coursera's MOOCs, MPI's, Dataslab OpenLab, and Velocity courses. Understands language learning, like etiquette and human language perception, across languages and cultural perspectives, is an important research area. Can you think of any obvious defense against this issue that involves a technical community? I ask because I suffer from community engineering paralysis. Thinking outside the box is hard, so I always try to take on projects and expertise in


  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 2 ========================================
 Learn more about the power of Go's neural network.

Go also has built-in support for text-based logic, thereby enabling applications that streamline complex algorithms by adding generic logic to plain text presentations.

Wired eyes served me well in college

In college, I used to cram the amount of books I read in two months into probably total books as many as 2000 in total. I acquired the ability to multitask and was able to analyze piles of data in very little time compared to other college students at the time and application developers in general.

Now that I have relocated to Florida,


  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 3 ========================================
 We are looking for junior engineers looking to get their foot in the door somewhere fascinating key installations engineering decision makers, software developers and data analysts...

Vancouver's first high-performance Deep Learning CRUD engine Supercomputer M23 (DLR22-K0)

18 May, 2017

Ukrainian researchers Nana Atemi he have shown what might be the world's first multicenter deep learning implementation: the implementation of extensive machine learning algorithms on heavyweight academic commercial products...

Canons 6.4 (Summer after Summer) - repository links for all the major improvements to Canonical GL Linus


100%|██████████| 125/125 [00:18<00:00,  7.20it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 4 ========================================
 Have a question? See our answers in the Answer Center. The Answers Plugin will ask you questions and ponder them over the course of a workweek. The only magic is your IQ, so wisdom speaks with your machine. Have a question or comment? Do you want to share your Watson Technologies Know-How(BTK) with the world? Use our new Facebook page. If you'd like to be investigated for breaking laws via cost and schedule, delete us from your Yahoo Groups. Or, connect with us on LinkedIn. I Pray, esp. Almighty God, an Eternity resting from everlasting glory. Amen. Frances:


In [53]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=125, seed=0, sample=True, top_p=0.9, nsamples=4)

  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Skip all the introductory materials we make available (get up to speed by reading about these areas). Take a Google or TechNet course on Machine Learning. Learn how to use Coursera's MOOCs, MPI's, Dataslab OpenLab, and Velocity courses. Understands language learning, like etiquette and human language perception, across languages and cultural perspectives, is an important research area. Can you think of any obvious defense against this issue that involves a technical community? I ask because I suffer from community engineering paralysis. Thinking outside the box is hard, so I always try to take on projects and expertise in


  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 2 ========================================
 Learn more about the power of Go's neural network.

Go also has built-in support for text-based logic, thereby enabling applications that streamline complex algorithms by adding generic logic to plain text presentations.

Wired eyes served me well in college

In college, I used to cram the amount of books I read in two months into probably total books as many as 2000 in total. I acquired the ability to multitask and was able to analyze piles of data in very little time compared to other college students at the time and application developers in general.

Now that I have relocated to Florida,


  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 3 ========================================
 We are looking for junior engineers looking to get their foot in the door somewhere fascinating key installations engineering decision makers, software developers and data analysts...

Vancouver's first high-performance Deep Learning CRUD engine Supercomputer M23 (DLR22-K0)

18 May, 2017

Ukrainian researchers Nana Atemi he have shown what might be the world's first multicenter deep learning implementation: the implementation of extensive machine learning algorithms on heavyweight academic commercial products...

Canons 6.4 (Summer after Summer) - repository links for all the major improvements to Canonical GL Linus


100%|██████████| 125/125 [00:18<00:00,  7.20it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 4 ========================================
 Have a question? See our answers in the Answer Center. The Answers Plugin will ask you questions and ponder them over the course of a workweek. The only magic is your IQ, so wisdom speaks with your machine. Have a question or comment? Do you want to share your Watson Technologies Know-How(BTK) with the world? Use our new Facebook page. If you'd like to be investigated for breaking laws via cost and schedule, delete us from your Yahoo Groups. Or, connect with us on LinkedIn. I Pray, esp. Almighty God, an Eternity resting from everlasting glory. Amen. Frances:


In [54]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=125, seed=0, sample=True, temperature=0.9, nsamples=3)

  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Answer our online quiz to learn more.

All of NLP's basic functions are defined by the Schema library, which has been proven to work as a family of intuitive models of Bayesian statistics, evolved from R and coupled to Phi Machine Learning.

GPio are applications, while machine learning is pioneered by the Google Brain and DeepMind AI teams.

The last name of the author is "Conrad".


This site uses cookies, if you would like to view the cookie details please visit www.cookiepolicy.org/party.php. There you will find all track of cookies,


  0%|          | 0/125 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 2 ========================================
 Improving production-ready marketing is one thing, but if you're looking to achieve better image search results, you'll want to spend time learning how to use machine learning and deep learning techniques. Have you researched and have any question? Click below to leave a comment and get in touch!

Image Credits: iStockphoto/romjones30<|endoftext|>


100%|██████████| 125/125 [00:17<00:00,  7.23it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 3 ========================================
 H1N1Partnerships#4

And now you hear laughter …<|endoftext|>


In [72]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=100, seed=0, sample=True, top_k=40, temperature=1.2)
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=500, seed=None, sample=True, top_k=40, temperature=1.2)

  0%|          | 0/500 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Feel free to fill out our online survey and make some friends. If you want to know about Google's upcoming project and more. Also feel free to send us an email with questions you may have.


Thanks for reading.<|endoftext|>


100%|██████████| 500/500 [01:26<00:00,  4.30it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================


No need to rush to register now, just keep reading!


What are Machine Learning and Natural Language Processing?


Many of the applications of machine learning and natural language processing are based on machine learning - a natural language approach for analysing objects or relationships that appear in large datasets in real time and which gives rise in this work to a number of fascinating applications and uses in areas as diverse as human-machine interaction systems, image editing to recognize natural human faces, speech synthesis for machine translation, web browsing and document creation, database management, real estate development, and so on and there is a lot more.


There is much to write about but let me start with this simple point from another article:

Linguistic analysis, or NLP (Natural language proc

In [68]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=100, seed=0, sample=True, top_k=40, temperature=1.2, nsamples=3)

  0%|          | 0/100 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Feel free to fill out our online survey and make some friends. If you want to know about Google's upcoming project and more. Also feel free to send us an email with questions you may have.


Thanks for reading.<|endoftext|>


  0%|          | 0/100 [00:00<?, ?it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 2 ========================================


No need to rush to register now, just keep reading!


What are Machine Learning and Natural Language Processing?


Many of the applications of machine learning and natural language processing are based on machine learning - a natural language approach for analysing objects or relationships that appear in large datasets in real time and which gives rise in this work to a number of fascinating applications and uses in areas as diverse as human-machine interaction systems, image editing to recognize natural human faces, speech synthesis for machine


100%|██████████| 100/100 [00:16<00:00,  6.32it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 3 ========================================
 Post a comment below to share your deep insights in the field :)

This post (Machine Learning/NNP/NLP Tutorial) is licensed under a Creative Commons Attribution 4.0 International License.<|endoftext|>


In [62]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=300, seed=0, sample=True, top_p=0.5)

100%|██████████| 300/300 [00:55<00:00,  5.25it/s]


PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Skip all the introductory materials we make available (get up to speed by reading about these areas). Take a Google or TechNet course on Machine Learning. Learn how to use Coursera's MOOCs, MPI's, Dataslab OpenLab, and Velocity courses. Understands language learning, like etiquette and human language perception, across languages and cultural perspectives, is an important research area. Can you think of any obvious defense against this issue that involves a technical community? I ask because I suffer from community engineering paralysis. Thinking outside the box is hard, so I always try to take on projects and expertise in fields that may be at distinct risk of extinction.

Full Video Those are below and here. What are the keys to mtn zero's success? They're equity & vehicle sentiment categories. Automotive appraisal

In [60]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=300, seed=0, sample=True, top_p=0.9)

100%|██████████| 300/300 [00:58<00:00,  2.59it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Skip all the introductory materials we make available (get up to speed by reading about these areas). Take a Google or TechNet course on Machine Learning. Learn how to use Coursera's MOOCs, MPI's, Dataslab OpenLab, and Velocity courses. Understands language learning, like etiquette and human language perception, across languages and cultural perspectives, is an important research area. Can you think of any obvious defense against this issue that involves a technical community? I ask because I suffer from community engineering paralysis. Thinking outside the box is hard, so I always try to take on projects and expertise in fields that may be at distinct risk of extinction.

Full Video Those are below and here. What are the keys to mtn zero's success? They're equity & vehicle sentiment categories. Automotive appraisal

In [61]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=300, seed=0, sample=True)

100%|██████████| 300/300 [01:03<00:00,  5.16it/s]


PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Skip all the introductory materials we make available (get up to speed by reading about these areas). Take a Google or TechNet course on Machine Learning. Learn how to use Coursera's MOOCs, MPI's, Dataslab OpenLab, and Velocity courses. Understands language learning, like etiquette and human language perception, across languages and cultural perspectives, is an important research area. Can you think of any obvious defense against this issue that involves a technical community? I ask because I suffer from community engineering paralysis. Thinking outside the box is hard, so I always try to take on projects and expertise in fields that may be at distinct risk of extinction.

Full Video Those are below and here. What are the keys to mtn zero's success? They're equity & vehicle sentiment categories. Automotive appraisal

In [55]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=125, seed=0, sample=True, top_k=20)

100%|██████████| 125/125 [00:19<00:00,  6.51it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================
 Tell us your answers on Twitter.

This article is part of an ongoing series of posts exploring the challenges of machine learning and natural language processing. Click here to see other posts on this topic.

Machine Learning and Natural Language Processing<|endoftext|>


In [57]:
run_model('What do you know about Machine Learning and Natural Language Processing?', 
          length=512, seed=1, sample=True, top_k=10)

100%|██████████| 512/512 [01:39<00:00,  4.24it/s]

PROMPT: What do you know about Machine Learning and Natural Language Processing?
======================================== SAMPLE 1 ========================================


I am an expert in artificial intelligence and natural language processing.

What is your background and expertise in Artificial Intelligence?

I am a graduate from MIT Media Lab where I worked on the machine learning team.

What is your favorite thing about artificial intelligence?

My favorite thing about machine learning is that it's so easy. It's so easy to understand. It's a very intuitive thing to understand. Machine learning is really easy to understand as well. If you understand machine learning, you'll understand how to apply it for different tasks, and that makes it so easy to learn how to use it.

How did you discover natural language processing (NLP) and how did you get into it?

I discovered NLP when I was working with a group of kids at a high school, and the teachers would ask them to type out some se

In [35]:
prompt = 'What do you know about Machine Learning and Natural Language Processing?'
batch_size = 1
length = 128
temperature = 1
top_k = 40
sample = False
seed = 0

np.random.seed(seed)
torch.random.manual_seed(seed)

context = torch.tensor(enc.encode(prompt), device=device, dtype=torch.long).unsqueeze(0).repeat(batch_size, 1)
prev = context
output = context
past = None
with torch.no_grad():
    for i in trange(length):
        logits, past = model(prev, past=past)
        logits = logits[:, -1, :] / temperature
        logits = top_k_logits(logits, k=top_k)
        log_probs = F.softmax(logits, dim=-1)
        if sample:
            prev = torch.multinomial(log_probs, num_samples=1)
        else:
            _, prev = torch.topk(logits, k=1, dim=-1)
        output = torch.cat((output, prev), dim=1)

out = output[:, context.shape[1]:].tolist()
print(prompt + '\n')
for i in range(batch_size):
    text = enc.decode(out[i])
    print(text)

100%|██████████| 128/128 [00:21<00:00,  6.04it/s]

What do you know about Machine Learning and Natural Language Processing?



I'm a software engineer and I'm currently working on a project called "Machine Learning for the Web". I'm also a big fan of natural language processing and I'm working on a project called "Natural Language Processing for the Web".

What's your favorite thing about machine learning?

I love the fact that it's so easy to use. I love the fact that it's so easy to use.

What's your favorite thing about natural language processing?

I love the fact that it's so easy to use. I love the fact that it's so easy to use.

What's


In [44]:
type(round(logits.shape[1] * 0.3))

int

In [52]:
seed = 0
np.random.seed(seed)
torch.random.manual_seed(seed)
run_model('What do you know about Machine Learning and Natural Language Processing?',
          length=128, sample=False)


100%|██████████| 128/128 [00:28<00:00,  5.02it/s]

======================================== SAMPLE 1 ========================================


I'm a software engineer and I'm currently working on a project called "Machine Learning for the Web". I'm also a big fan of natural language processing and I'm working on a project called "Natural Language Processing for the Web".

What's your favorite thing about machine learning?

I love the fact that it's so easy to use. I love the fact that it's so easy to use.

What's your favorite thing about natural language processing?

I love the fact that it's so easy to use. I love the fact that it's so easy to use.

What's


In [12]:
seed = 0
for seed in range(10):
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    run_model('What do you know about Machine Learning and Natural Language Processing?')

  0%|          | 0/512 [00:00<?, ?it/s]

======================================== SAMPLE 1 ========================================
 Skip all the introductory materials we make available (get up to speed by reading about these areas). Take a Google or TechNet course on Machine Learning. Learn how to use Coursera's MOOCs, MPI's, Dataslab OpenLab, and Velocity courses. Understands language learning, like etiquette and human language perception, across languages and cultural perspectives, is an important research area. Can you think of any obvious defense against this issue that involves a technical community? I ask because I suffer from community engineering paralysis. Thinking outside the box is hard, so I always try to take on projects and expertise in fields that may be at distinct risk of extinction.

Full Video Those are below and here. What are the keys to mtn zero's success? They're equity & vehicle sentiment categories. Automotive appraisal is a powerful part of property valuation. Our results showed that we can convert

  0%|          | 0/512 [00:00<?, ?it/s]

======================================== SAMPLE 1 ========================================


In this article, you'll learn about…

How to use Machine Learning and Natural Language Pair Questions (MLPQ tracks), and how to train Machine Learning and Nanosense on SPSS data.

When I think about results in GPU trained data that can automatically help me diagnose a problem, it often feels like I am seeing the results of new movies. What makes a world outside of graphics surprising and uncanny?
 (See also: Expectations, Perception You Want Machine Learning to Discover).

• Objective . Knowing where promises of high results lie and what new strategies they imply

• Validation . Finding demand side proofs of upper bound validation of machine learning

• Alternatives. When and how we can quantify these computations

• Results. How in fact machine learning and natural language processing to uncover relevant methodologies

• Money question – What can other members of the family help me?

Implement

  0%|          | 0/512 [00:00<?, ?it/s]

...

======================================== SAMPLE 1 ========================================
 We want to hear about your natural language processing experiences!

Banshee, Fog, Mace Khanana and Friman Wars Based out of Oslo, this crew works on the least of these and want you to know it! Learn more about Banhafis castle here – facebook

Botkick Jul 26

NGO asking for help for web service that no Social Media management company can investigate should anyone presume: hardware or software? The member's main concern is that both technologies are so prevalent in today's society in wide range of situations that most IT platforms don't even bother linking or formatting their entire B2S packages.

Reek, Emperor

1865 Tools for Browalscience published by Jerry Golomb on Inset Security Magazine "These tools to myself find solutions for this software "software lookup" for backend control jobs, is it better to rely on base/formatting away, or to manually style at RBbenko's Treatante machine or use Al

  0%|          | 0/512 [00:00<?, ?it/s]

======================================== SAMPLE 1 ========================================
 Have you used it when you're a teacher or intern in the classroom? Let's discuss it! Ask forth your questions in the comments below, or tweet me at 👍 @MichaelLadd. Search for him on GitHub. Related<|endoftext|>China's huge black arrows on a map on view inside the three-story box of a giant advances department store named China Ballo, which sells entertainment jewelry starting at 7,500 yuan ($8,700) a pop. (Bao Yu/CQ Roll Call)

Switches and alternates hang on colorful pillows and pumps above hands that wobble in bedded undies as they hum and echo underwater images by Blizzard Entertainment's redrawing of video games as magical theaters now play imagination from South Korea. Boys play with dot-matrix petals on face masks decorated with big image bubbles, when hoops shoot from slats in blond fairy swimwear modeled after those worn by the country's Olympic athletes. Mouse , duck and cat eyes sparkl

100%|██████████| 512/512 [01:41<00:00,  4.13it/s]

======================================== SAMPLE 1 ========================================



If you are interested to know more about AI/NCP please I do offer tutorials on several popular subjects like deep learning framework, deep learning network architectures, Neural Networks and more online.


There are a lot of options available if you are interested in AI Programming.


I offer two tutorials where you can visit this beautiful 200 square feet facility where we teach you basic Machine Learning principles like recursion, implementation, learning rates, reconstruction, trained HML files and more.


LDP is provided by HealthLine as we do right now everything through email to help. Please don't hesitate to ask any questions you might have during any of our trainings or train your brain to recall something.


Techniques and activities include copious tutorials such as green eyes. Deep learning programming as A/B testing gmc$UmbRnBiBiQ QT[end automated natural language processing steps 

In [24]:
seed = 0
for seed in range(10):
    np.random.seed(seed)
    torch.random.manual_seed(seed)
    run_model('What do you know about Machine Learning and Natural Language Processing?', length=128)

  0%|          | 0/128 [00:00<?, ?it/s]

======================================== SAMPLE 1 ========================================
 Isn't it remarkable how powerful it is to let your mind run free of everything that's changing around it?

We've worked at Machine Learning Hub for more than 20 years and have discovered that as a beginning along the way we are primed for processes that are indistinguishable from human behavior.

Our first example premiered at the exact moment in history when Norma Bolt, a templated data scientist with the former Company That Builds Everything, won first prize to shut her down. One year later Bolt decided to give Carlos Miller fiscal incentives to write an article on Machine Learning and left working on it (besides the Ask Me Anything


  1%|          | 1/128 [00:00<00:16,  7.85it/s]

======================================== SAMPLE 1 ========================================


If you're a teacher or teacher person and you take the time to learn more about Machine Learning Prediction Python written by three people smoking cigarettes: VThisuppolo Stefano aka 'teacher' and Asundri 'magic' Alibato

Published – May 22, 2018

PETROSOFT THUNDER

Artistic Attorney is one of the recent founders

developed by Pierre Tonkin Architects for the Jesuit university of Sepho on the outskirts of Groningen. The 42 year old TorboÍso Pascal reflects particularly on the process behind this project:

Pierre has been true to his


  1%|          | 1/128 [00:00<00:17,  7.27it/s]

...

======================================== SAMPLE 1 ========================================


Machine Learning engineers simply use Translate which is a data-rich algorithm, crafted to automate mathematical verification and store tens of billions of data streams in a secure database.

YOUR GLOBAL LEADER

How did your time at University Connect image manipulation workshop effort impact your skill set?

At Baccalaureate I found that we prioritized knowledge over experience & mentorship tenure, leadership, actual work, and keep it real. In addition to our many ancestors, our knowledgeable Chinese and Vietnamese colleagues learn through field education & talent exchange—contorically crafts that empower us to be as trustworthy as we


  1%|          | 1/128 [00:00<00:16,  7.89it/s]

======================================== SAMPLE 1 ========================================
 Let me know in the comments below!

Image credits: Wikimedia Commons , dboven, GPSImagesMagnet.com, iLibrary http://tsk<|endoftext|>NEW DELHI: India is facing a serious crisis over what can happen when it needs to cull 2,500 trainees once it stalks express train with a massive explosive discovered here late last month.

Experts say constraints imposed on trainmen by railway authorities and lawmakers have allowed the project to fail, leading to an "inability" to get train utilised in the next couple of weeks.

Officials point out that intrusive screening of element 500 used in Pakistan


100%|██████████| 128/128 [00:08<00:00, 15.26it/s]

======================================== SAMPLE 1 ========================================


Allen: Well, I first wrote about understanding machine learning after a really cold RSS feed that I've seen to the nearest 100,000 on Slack. Sort of back to hacker stuff. I've learned a lot about language driven AI through my experiences with AI modelling, how the big pin backpacks really work. Also, understanding the bigger problems is difficult but worth it so far. If you can print an entirely pre-programmed neural model and it turns out you can do pre-code which is interesting to know what you're doing with that sensor on your TV that the AI is chasing for.

And so claiming
